In [1]:
import os
import sys
import importlib

import pandas as pd
import numpy as np

from scipy.io import mmread

sys.path.append(os.path.abspath("./src"))

import utility as ut
import hessenberg as hg
from variables import *
from qr import QR


pd.set_option("display.max_columns", 10)
pd.set_option("display.width", 1000)

print("Setup compelete.")


Setup compelete.


# Hessenberg Transform

## Complex Matrices

In [2]:
a = -20
b = 50
n = 5
m = ut.complex_matrix(n, a, b)
print(f"Original matrix:\n {pd.DataFrame(m)}")
h, u = hg.hessenberg_transform(m) 
print(f"Hessenberg transformed:\n {pd.DataFrame(h)}")
print(f"Transformation matrix:\n {pd.DataFrame(u)}")
print(f"Is the transformation similar: {np.allclose(u @ h @ u.conj().T - m, np.zeros((n, n)))}")

Original matrix:
                       0                     1                     2                     3                     4
0  -4.804923+23.730160j -16.359371+26.383236j  46.926128+26.718261j  -1.867321+49.754256j   5.114251+11.338629j
1   33.838611+0.862580j  12.626639+39.500811j  9.9135270+3.8399050j   3.431236-19.644613j  39.449039+42.764978j
2  41.628379-16.588279j   2.076414+12.034994j   2.040094+17.755369j -15.512318-15.657989j  -11.083593+2.843183j
3 -11.208934+34.592834j   2.503769-12.729379j -19.482661-13.254690j  10.762023-11.747013j  21.511528+34.422293j
4  44.704343-14.674559j  10.878339+10.881410j  10.786717-14.806481j  43.693147+42.332062j  40.518210+46.571637j
Hessenberg transformed:
                      0                     1                     2                     3                     4
0 -4.804923+23.730160j  -6.614516-12.330768j -41.523079+31.746777j  -22.771283-5.375003j -43.767095+33.504028j
1 -81.765944-2.084296j  28.811811+33.450940j  24.279457-26.5811

## Real Matrices

### Random Matrices

In [3]:
a = -20
b = 50
n = 5
m = (b - a) * np.random.default_rng().random((n, n)) + a
print(f"Original matrix:\n {pd.DataFrame(m)}")
h, u = hg.hessenberg_transform(m) 
print(f"Hessenberg transformed:\n {pd.DataFrame(h)}")
print(f"Transformation matrix:\n {pd.DataFrame(u)}")
print(f"Is the transformation similar: {np.allclose(u @ h @ u.T - m, np.zeros((n, n)))}")

Original matrix:
            0          1          2          3          4
0  26.248482  26.035779  12.983198  36.592658  14.641317
1  28.080119  49.682225  17.964609  38.623893  25.893820
2 -13.639360  15.901645  48.963673  -4.781431  13.309064
3  34.349359 -18.450561  33.758003  12.629556 -13.537496
4  -3.270996  32.452370  40.302688 -13.609501  -1.622084
Hessenberg transformed:
            0          1          2          3          4
0  26.248482 -37.890019   4.059151 -24.649395 -18.441612
1 -46.530668  25.939384 -15.871055  12.113619  24.290271
2   0.000000  46.640610  23.838191  31.696676  -5.029004
3   0.000000   0.000000  33.152324  24.018651  57.396180
4   0.000000   0.000000   0.000000  28.295707  35.857144
Transformation matrix:
      0         1         2         3         4
0  1.0  0.000000  0.000000  0.000000  0.000000
1  0.0 -0.603476 -0.766597 -0.115069 -0.186830
2  0.0  0.293126  0.034692 -0.290635 -0.910167
3  0.0 -0.738209  0.641149 -0.111172 -0.177808
4  0.0  0.0702

### Matrix Market

In [4]:
files = ["west0381", "blckhole"]
for file in files:
	mat = mmread(os.path.join("./test_matrices", ".".join([file, MATRIX_MARKET_FILE_EXT])))
	m = mat.toarray()
	h, u = hg.hessenberg_transform(m) 
	print(f"Is the transformation similar: {np.allclose(u @ h @ u.T - m, np.zeros((m.shape[0], m.shape[0])))}")

Is the transformation similar: True
Is the transformation similar: True


# QR

## Wilkinson Shift

### Complex Matrices

In [27]:
a = -20
b = 50
n = 5
atol = 1e-8
m = ut.complex_matrix(n, a, b)
qr_alg = QR(m)
u, r = qr_alg.qr_wilkinson_shift(1e-128, 500)
eigs = np.sort(np.linalg.eig(qr_alg.H.astype(np.complex128))[0])[::-1]
eigs_extracted = np.sort(qr_alg.extract_eigs(r))[::-1]
print(f"{pd.DataFrame(eigs, columns = ['Eigenvalues (Numpy)'])}")
print(f"{pd.DataFrame(eigs_extracted, columns = ['Eigenvalues (Script)'])}")
print(f"Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance {atol}: {np.allclose(eigs_extracted, eigs, rtol = 0, atol = atol)}")

     Eigenvalues (Numpy)
0  56.209983+100.299996j
1  36.2441470+8.0821680j
2   32.049115-39.880043j
3 -0.929528+046.0506460j
4  -30.748723-10.187563j
    Eigenvalues (Script)
0  56.209983+100.299996j
1  36.2441470+8.0821680j
2   32.049115-39.880043j
3 -0.929528+046.0506460j
4  -30.748723-10.187563j
Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance 1e-08: True


### Real Matrices

#### Random Matrices

In [28]:
a = -20
b = 50
n = 5
atol = 1e-8
m = (b - a) * np.random.default_rng().random((n, n)) + a
qr_alg = QR(m)
u, r = qr_alg.qr_wilkinson_shift(1e-128, 500)
eigs = np.sort(np.linalg.eig(qr_alg.H.astype(np.complex128))[0])[::-1]
eigs_extracted = np.sort(qr_alg.extract_eigs(r))[::-1]
print(f"{pd.DataFrame(eigs, columns = ['Eigenvalues (Numpy)'])}")
print(f"{pd.DataFrame(eigs_extracted, columns = ['Eigenvalues (Script)'])}")
print(f"Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance {atol}: {np.allclose(eigs_extracted, eigs, rtol = 0, atol = atol)}")

    Eigenvalues (Numpy)
0   69.190598-0.000000j
1   3.283694+14.869563j
2   3.283694-14.869563j
3 -31.836209+23.733588j
4 -31.836209-23.733588j
   Eigenvalues (Script)
0   69.190598+0.000000j
1   3.283694+14.869563j
2   3.283694-14.869563j
3 -31.836209+23.733588j
4 -31.836209-23.733588j
Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance 1e-08: True


In [29]:
m = np.array([[7, 3, 4, -11, -9, -2],
     [-6, 4, -5, 7, 1, 12],
     [-1, -9, 2, 2, 9, 1],
     [-8, 0, -1, 5, 0, 8],
     [-4, 3, -5, 7, 2, 10],
     [6, 1, 4, -11, -7, -1]], dtype = np.float64)
atol = 1e-8
qr_alg = QR(m)
u, r = qr_alg.qr_wilkinson_shift(1e-128, 500)
eigs = np.sort(np.linalg.eig(qr_alg.H.astype(np.complex128))[0])[::-1]
eigs_extracted = np.sort(qr_alg.extract_eigs(r))[::-1]
print(f"{pd.DataFrame(eigs, columns = ['Eigenvalues (Numpy)'])}")
print(f"{pd.DataFrame(eigs_extracted, columns = ['Eigenvalues (Script)'])}")
print(f"Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance {atol}: {np.allclose(eigs_extracted, eigs, rtol = 0, atol = atol)}")

   Eigenvalues (Numpy)
0             5.0+6.0j
1             5.0-6.0j
2             4.0+0.0j
3             3.0+0.0j
4             1.0+2.0j
5             1.0-2.0j
   Eigenvalues (Script)
0              5.0+6.0j
1              5.0-6.0j
2              4.0+0.0j
3              3.0+0.0j
4              1.0+2.0j
5              1.0-2.0j
Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance 1e-08: True


#### Matrix Market

In [42]:
files = ["gre__115"]
for file in files:
	mat = mmread(os.path.join("./test_matrices", ".".join([file, MATRIX_MARKET_FILE_EXT])))
	m = mat.toarray()
	atol = 1e-6
	qr_alg = QR(m)
	u, r = qr_alg.qr_wilkinson_shift(1e-64, 100)
	eigs = np.sort(np.linalg.eig(qr_alg.H)[0])[::-1]
	eigs_extracted = np.sort(qr_alg.extract_eigs(r))[::-1]
	# print(f"eigs real:\n {pd.DataFrame(eigs_real)}")
	# print(f"eigs approximated:\n {pd.DataFrame(eigs_extracted)}")
	print(f"Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance {atol}: {np.allclose(eigs_extracted, eigs, rtol = 0, atol = atol)}")
	mismatched_elements = ~np.isclose(eigs_extracted, eigs, rtol = 0, atol = atol)
	
	with open("output_mm.txt", "w") as f:
		f.write(f"{pd.DataFrame(np.vstack([eigs_real[mismatched_elements], eigs[mismatched_elements]]).T, columns = ['Eigenvalues (Numpy)', 'Eigenvalues (Script)']).to_string()}")
	# with open("output_mm.txt", "w") as f:
	# 	f.write(f"{pd.DataFrame(np.vstack([eigs_real, eigs_extracted, eigs_real - eigs_extracted]).T, columns = ['Eigenvalues (Numpy)', 'Eigenvalues (Script)', 'Difference']).to_string()}")

Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance 1e-08: False


## Double Shift

In [49]:
m = np.array([[7, 3, 4, -11, -9, -2],
     [-6, 4, -5, 7, 1, 12],
     [-1, -9, 2, 2, 9, 1],
     [-8, 0, -1, 5, 0, 8],
     [-4, 3, -5, 7, 2, 10],
     [6, 1, 4, -11, -7, -1]], dtype = np.float64)

qr_alg = QR(m)
u, r = qr_alg.double_shift(1e-128, 500)
# print(f"{pd.DataFrame(r)}")
eigs = np.sort(np.diag(r))[::-1]
eigs_real = np.sort(np.linalg.eig(qr_alg.H)[0])[::-1]
eigs_extracted = np.sort(qr_alg.extract_eigs(r))[::-1]
print(f"eigs real:\n {pd.DataFrame(eigs_real)}")
print(f"eigs approximated:\n {pd.DataFrame(eigs_extracted)}")

ValueError: M must be square and of shape (2, 2) but given shape is (3, 3).